In [1]:
import pandas as pd

1. Load Dataset
===============

In [3]:
train = pd.read_csv('./input_titanic/train.csv', index_col='PassengerId')
print(train.shape)
#print(train.info())
train.head(2)

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [4]:
test = pd.read_csv('./input_titanic/test.csv', index_col='PassengerId')
print(test.shape)
test.head(2)

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


2.데이터 전처리(PreProcessing)
===========================

In [5]:
# 성별(Sex) 값이 male은 0으로, female은 1로 변환
train.loc[train['Sex'] == 'male','Sex'] = 0
train.loc[train['Sex'] == 'female','Sex'] = 1

In [6]:
train['Sex'].value_counts()

0    577
1    314
Name: Sex, dtype: int64

In [7]:
test.loc[test['Sex'] == 'male','Sex'] = 0
test.loc[test['Sex'] == 'female','Sex'] = 1

In [8]:
test['Sex'].value_counts()

0    266
1    152
Name: Sex, dtype: int64

2.2 Fill in Missing Value
==========================

In [9]:
# Fare(요금)가 null 인 값을 확인
train.loc[train['Fare'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,


In [10]:
test.loc[test['Fare'].isnull(),'Fare'] = 0
test.loc[test['Fare'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,


2.3 One Hot Encoding
====================

In [11]:
train['Embarked_C'] = train['Embarked'] == 'C'
train['Embarked_S'] = train['Embarked'] == 'S'
train['Embarked_Q'] = train['Embarked'] == 'Q'
train[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']].head()

,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
1,S,False,True,False
2,C,True,False,False
3,S,False,True,False
4,S,False,True,False
5,S,False,True,False


In [12]:
test['Embarked_C'] = test['Embarked'] == 'C'
test['Embarked_S'] = test['Embarked'] == 'S'
test['Embarked_Q'] = test['Embarked'] == 'Q'
test[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']].head()

,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
892,Q,False,False,True
893,S,False,True,False
894,Q,False,False,True
895,S,False,True,False
896,S,False,True,False


3. Train
========

3.1 Feature Enginnering
=======================

In [13]:
#필요한 컬럼을 선택
feature_names = ['Sex','Fare','Pclass','Embarked_C','Embarked_S','Embarked_Q']
feature_names

['Sex', 'Fare', 'Pclass', 'Embarked_C', 'Embarked_S', 'Embarked_Q']

In [14]:
# fit() 할 훈련 데이터 문제를 만듬
X_train = train[feature_names]
print(X_train.shape)
X_train.head()

(891, 6)


,Sex,Fare,Pclass,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
1,0,7.2500,3,False,True,False
2,1,71.2833,1,True,False,False
3,1,7.9250,3,False,True,False
4,1,53.1000,1,False,True,False
5,0,8.0500,3,False,True,False


In [15]:
X_test = test[feature_names]
print(X_test.shape)
X_test.head()

(418, 6)


,Sex,Fare,Pclass,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
892,0,7.8292,3,False,False,True
893,1,7.0000,3,False,True,False
894,0,9.6875,2,False,False,True
895,0,8.6625,3,False,True,False
896,1,12.2875,3,False,True,False


In [16]:
#훈련데이터의 레이블(답) 생성
#train DataFrame에서 Survived 컬럼을 가져와서 y_train에 저장
label_name='Survived'
y_train = train[label_name]
print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

3.2 Decision Tree(결정트리) 알고리즘 선택해서 학습하기
==============================================

In [34]:
# DecisionTreeClassifier 알고리즘 선택
#from sklearn.tree import DecisionTreeClassifier
#import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [35]:
#의사 결정트리의 최대깊이를 5로 설정
#model = DecisionTreeClassifier(max_depth=5,random_state=0)
#model
#model  = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
#model
model = RandomForestClassifier(n_estimators=100,max_depth=9,min_samples_split=6, min_samples_leaf=4)
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [36]:
#학습하기
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
#예측하기
predictions = model.predict(X_test)
print(predictions.shape)
predictions

(418,)


array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

4. Submit 하기
=============

In [41]:
submit = pd.read_csv('./input_titanic/gender_submission.csv', index_col='PassengerId')

In [42]:
print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [43]:
#submit dataframe의 Survived 컬럼을 predictions(예측한값)으로 변경
submit['Survived'] = predictions
print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


### Submit 할 Survived 컬럼을 예측한 값으로 변경을 해야 함

In [33]:
# submit DataFrame을 csv 파일로 저장하기
submit.to_csv("./input_titanic/titanic_predict_00.csv")